In [184]:
#import numpy as np
#import matplotlib.pyplot as plt
#from ipywidgets import interactive
#%matplotlib inline


In [185]:
#from google.colab import drive
#drive.mount('/content/drive')

In [186]:
!pip install streamlit -q

In [187]:
import streamlit as sl

In [188]:

import pandas as pd
import numpy as np
import csv

In [189]:

movies=pd.read_csv("movie.csv")

In [190]:
movies.head(50)

In [191]:
movies.rename(columns = {'id':'movie_id'}, inplace = True)
movies["keywords"][0]

In [192]:

movies["tagline"][0]

In [193]:
movies["overview"][0]

In [194]:
df2=pd.read_csv("movie_credit.csv")

In [195]:
df2.head()

In [196]:

df2["cast"]

In [197]:
movies=movies.merge(df2,on="title")

In [198]:

movies.info()

In [199]:
import ast
def str_list(x):
  x=ast.literal_eval(x)
  l=[]
  for item in x:

    l.append(item['name'])

  return l

In [200]:
movies["cast"]=movies["cast"].apply(str_list)

In [201]:
import ast
def str2_list(x):
  x=ast.literal_eval(x)
  l=[]
  for item in x:
    if item['job']=="Director":
      l.append(item['name'])

  return l

In [202]:
movies["crew"]=movies["crew"].apply(str2_list)

In [203]:
movies["cast"]

In [204]:
movies["crew"]

In [205]:
movies["genres"]=movies['genres'].apply(str_list)
movies["keywords"]=movies['keywords'].apply(str_list)

In [206]:
movies.isna().sum()

In [207]:
movies.sample(5)

In [208]:
# removing space
def _join(x):
  l=[]
  for i in x:
    l.append(i.replace(" ",""))
  return l


In [209]:
movies["cast"]=movies["cast"].apply(_join)

In [210]:
movies["cast"]=movies["cast"].apply(lambda i:i[0:3])

In [211]:
movies["cast"]


In [212]:
new_df=pd.DataFrame(movies,columns=['id','homepage','title','cast','crew','overview','genres','keywords'])

In [213]:

new_df.head(5)

In [214]:
new_df['crew']=new_df['crew'].apply(_join)

In [215]:

new_df.head()

In [216]:
new_df.isna().sum()

In [217]:

new_df["homepage"]=new_df["homepage"].fillna(0)

In [218]:
url=list(enumerate(new_df["homepage"]))
new_df["urls"]=0
for i in range(len(url)):
  if new_df["homepage"][i]==0:

    x=("{}/{}".format("https://www.",str("".join(list(new_df["title"][url[i][0]].split())))))

    new_df["urls"][i]=x
  else:
    new_df["urls"][i]=new_df["homepage"][i]

In [219]:


# def link(url,i):
#   s=url
#   x="{} + {}".format("https://www.",str(new_df.iloc[[url[i][0]]].title))
#   iF s is "link"
#     return x
#   else:
#     return s
# # new_df[new_df["homepage"].fillna("{} + {}".format("https://www.",new_df["title"]]))
new_df["urls"]

In [220]:
# new_df.iloc[[0]].title
new_df.sample(5)

In [221]:
# to convert string into list of strings
def spliting(text):
  text=str(text).split()
  return text

In [222]:

new_df['overview']=new_df['overview'].apply(spliting)

In [223]:

new_df['all_tags']=new_df['overview'] + new_df['genres'] + new_df['keywords'] + new_df['cast'] + new_df['crew']

In [224]:

new_df.drop(columns=['overview','genres' ,'keywords','cast' , 'crew','homepage'],inplace=True)


In [225]:

new_df.head()

In [226]:

#converting all tags in lower case and deletion similarities

In [227]:
def convert_lower(text):
  l=[]
  for item in text:
    l.append(item.lower())
  return l

In [228]:
new_df['all_tags']=new_df['all_tags'].apply(convert_lower)

In [229]:

new_df.head()

In [230]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steming(text):
  l=[]
  for i in text:

    l.append(ps.stem(i))
  return l

In [231]:
new_df['all_tags']=new_df['all_tags'].apply(steming)

In [232]:
new_df.sample(5)

In [233]:
new_df["all_tags"]=new_df["all_tags"].apply(_join)

In [234]:
new_df['all_tags'] = new_df['all_tags'].apply(lambda x: " ".join(x))

In [235]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [236]:
data = cv.fit_transform(new_df['all_tags']).toarray()
data

In [237]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(data)
similarity

In [238]:
def movie_recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:8]:


      print("{} {}".format(new_df.iloc[i[0]].title,new_df.iloc[i[0]].urls))

In [239]:
movie_recommend("Avatar")

In [240]:
movie_recommend("Batman")

In [241]:
movie_recommend("2016: Obama's America")

In [242]:
movie_recommend("Iron Man")

In [243]:
import pickle

In [244]:

pickle.dump(new_df.to_dict,open("movie_recm.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))

In [245]:

np.array(new_df["title"])

In [246]:
#!pip install pyngrok

In [247]:
#!streamlit run app.py & npx localtunnel --port[80]